---
title: "Long-Term Development Indicators and Global Trends"
author: 
  - name: "Jacky An (Emory ID: 2486597)"
  - name: "Bernice (Emory ID: 2487091)"
  - name: "Eric"
date: "04/21/2025"
format: html
editor: 
  markdown: 
    wrap: 72
---


# Introduction

Our final project explores the long-term trends and interrelationships between key development indicators across countries using data from the World Bank’s World Development Indicators (WDI) database. We focus on life expectancy, fertility rate, population growth, school enrollment, and GDP per capita growth—indicators that collectively reflect the social, demographic, and economic development of nations between 1970 and 2023.

Using these variables, we examine how health, education, and demographic factors evolve over time and interact with one another across countries with different income levels, development statuses, and continental contexts. This includes evaluating how changes in primary school enrollment affect not only fertility and life expectancy but also broader outcomes like economic growth.

## Background

Development indicators are critical tools for understanding how
societies progress over time. Measures such as life expectancy and
fertility rate reveal how access to healthcare, education, and economic
opportunities impact the quality of life and long-term population
dynamics. In contrast, metrics like school enrollment and population
growth highlight the importance of human capital and demographic
pressure in shaping a country’s future trajectory.

Longitudinal data allows us to move beyond snapshots and instead capture
how these dynamics unfold over decades. For example, countries with
declining fertility and increasing school enrollment often exhibit
longer life expectancies, but this relationship may vary depending on
region, income level, or development stage.

The World Development Indicators (WDI) database provides extensive
historical data on these themes, making it an ideal resource for
studying global development trends over time. Our project leverages this
resource to analyze patterns from 1960 through 2023 across over 100
countries.

## Research Questions

Our project addresses the following questions:

1. How has life expectancy evolved globally since 1960, and how is it associated with fertility and education trends?

2. What is the relationship between school enrollment and fertility rate across countries?

3. Do countries with rapid population growth face distinct development challenges in terms of life expectancy and health outcomes?

4. How do demographic and education indicators jointly predict life expectancy, and does this relationship differ by region or development status?

5. What patterns emerge when comparing development indicators between developed and developing nations?

6. Is there a consistent relationship between primary school enrollment growth and GDP growth, particularly when segmented by income group?



## Objectives

The primary objectives of this project are to:

- Analyze long-term trends in life expectancy, fertility, population growth, school enrollment, and GDP growth using the World Development Indicators dataset.

- Examine the interrelationships among key development indicators from multiple comparative angles: by income group, by development status, and by continent.

- Use regression models and correlation analysis to explore how demographic and educational factors contribute to both life expectancy and economic growth.

- Investigate whether countries with extremely high school enrollment rates (e.g., >100% gross) exhibit unique demographic or economic profiles.

- Visualize development trajectories using clear and informative visualizations, including heatmaps, facet scatter plots, and regression overlays.

## Data and Methods

This project uses four key indicators from the WDI:

-   **Life Expectancy at Birth (years)** — `SP.DYN.LE00.IN`\
-   **Fertility Rate (births per woman)** — `SP.DYN.TFRT.IN`\
-   **Population Growth (annual %)** — `SP.POP.GROW`\
-   **School Enrollment, Primary (% gross)** — `SE.PRM.ENRR`
-   **GDP per capita (current US$)** - `NY.GDP.PCAP.CD`


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Load and clean data
df = pd.read_csv("datasets/wdi_data.csv")
df = df[df["Country Name"].notna()]
year_cols = [col for col in df.columns if "YR" in col]

df_long = df.melt(
    id_vars=["Country Name", "Country Code", "Series Name", "Series Code"],
    value_vars=year_cols,
    var_name="Year", value_name="Value"
)

df_long["Year"] = df_long["Year"].str.extract(r"(\d{4})")
df_long = df_long[df_long["Year"].notna()]
df_long["Year"] = df_long["Year"].astype(int)


df_long = df_long[df_long["Value"].notna() & (df_long["Value"] != "..")]

df_wide = df_long.pivot(
    index=["Country Name", "Country Code", "Year"],
    columns="Series Name",
    values="Value"
).reset_index()

df_wide = df_wide.rename(columns={
    "Life expectancy at birth, total (years)": "LifeExpectancy",
    "Fertility rate, total (births per woman)": "FertilityRate",
    "Population growth (annual %)": "PopGrowth",
    "School enrollment, primary (% gross)": "PrimaryEnrollGross"
})

for col in ["LifeExpectancy", "FertilityRate", "PopGrowth", "PrimaryEnrollGross"]:
    df_wide[col] = pd.to_numeric(df_wide[col], errors="coerce")

## Summary Statistics


In [ ]:
df_wide[["LifeExpectancy", "FertilityRate", "PopGrowth", "PrimaryEnrollGross"]].describe()

These summary statistics help us understand the general range and
variability of the four indicators across time and space. Life
expectancy shows a wide range across countries, from below 50 years to
over 80. Fertility rate and population growth also exhibit significant
variance, reflecting different stages of demographic transition.

## Visualization: Life Expectancy Over Time


In [ ]:
plt.figure(figsize=(12, 6))
selected = ["United States", "India", "China", "Nigeria"]
for country in selected:
    sns.lineplot(
        data=df_wide[df_wide["Country Name"] == country],
        x="Year", y="LifeExpectancy", label=country
    )
plt.title("Life Expectancy Over Time (1960–2023)")
plt.ylabel("Life Expectancy (Years)")
plt.xlabel("Year")
plt.legend()
plt.tight_layout()
plt.show()

This chart clearly shows the upward trajectory of life expectancy across
all selected countries, albeit with different starting points and rates
of change.

## Exploring Relationships Between Indicators

We now use a pairplot to explore the relationships among the four
indicators:


In [ ]:
sns.pairplot(
    df_wide[["LifeExpectancy", "FertilityRate", "PopGrowth", "PrimaryEnrollGross"]].dropna()
)

This plot reveals a strong negative correlation between life expectancy
and fertility, and a weak positive association between life expectancy
and school enrollment.

# Regression Analysis

We build a linear regression model to predict life expectancy using
fertility, population growth, and school enrollment:


In [ ]:
ols_model = smf.ols(
    "LifeExpectancy ~ FertilityRate + PopGrowth + PrimaryEnrollGross",
    data=df_wide
).fit()
ols_model.summary()

Interpretation Fertility Rate shows a strong negative association with
life expectancy, suggesting that lower fertility correlates with longer
lives.

School Enrollment has a modest positive effect on life expectancy,
highlighting the role of education in improving population health.

Population Growth shows mixed or negligible influence, depending on
regional factors.

## Additional Insights: Country Group Analysis

We can group countries by income level or region (if data permits) to
further refine our analysis. For now, we use a simple example of
comparing high vs low fertility countries:


In [ ]:
df_wide["FertilityGroup"] = pd.qcut(
    df_wide["FertilityRate"],
    q=2,
    labels=["Low", "High"]
)

sns.boxplot(data=df_wide, x="FertilityGroup", y="LifeExpectancy")
plt.title("Life Expectancy by Fertility Group")
plt.xlabel("Fertility Rate Group")
plt.ylabel("Life Expectancy")
plt.show()

The boxplot clearly shows that countries with lower fertility tend to
have higher life expectancy.

## Conclusion

Our analysis shows that demographic and education indicators are deeply
intertwined with population health. Life expectancy has improved
dramatically worldwide since 1960, particularly in countries with
declining fertility and improved school enrollment.

Multivariable regression confirms the strong influence of fertility rate
on life expectancy, with additional positive effects from educational
access. While population growth plays a less clear role, it may be
relevant when considered in combination with economic or geographic
context.

This project reinforces the importance of investing in education and
healthcare to foster long-term development and improve quality of life
globally.

------------------------------------------------------------------------